# Event Extraction (Chunked Processing)

This notebook takes the full raw Kaggle dataset (`reddit_opinion_ru_ua.csv`, ~5GB) and extracts
five smaller event-based datasets for analysis.

Goals:
- Avoid memory issues by using chunked loading
- Filter comments into predefined event windows
- Keep **all** subreddits but label whether each comment is from a U.S.-focused subreddit
- Save one CSV per event for fast downstream analysis (sentiment + topics + Streamlit)

Outputs (created locally):
- `data/processed/event1_kyiv.csv`
- `data/processed/event2_kherson.csv`
- `data/processed/event3_stalemate.csv`
- `data/processed/event4_trump_election.csv`
- `data/processed/event5_white_house_meeting.csv`


## 1. Setup

We load required libraries and define file paths.

Because the source file is very large, we will read it in chunks using pandas `chunksize`.


In [23]:
import os
import pandas as pd
from collections import defaultdict


In [25]:
# Input (raw 5GB Kaggle file)
file_path = "../data/raw/reddit_opinion_ru_ua.csv"

# Output directory (event files will be created here)
output_dir = "../data/processed"
os.makedirs(output_dir, exist_ok=True)

# Chunk size (rows per chunk)
chunksize = 200_000


## 2. Define Event Windows

These are the five time windows we will extract from the full dataset.
Each window represents a major moment in the Ukraine war or U.S. political narrative.


In [45]:
event_windows = {
    "event1_kyiv": ("2022-02-20", "2022-03-20"),
    "event2_kherson": ("2022-10-15", "2022-12-01"),
    "event3_stalemate": ("2023-03-01", "2023-06-30"),
    "event4_trump_election": ("2024-10-25", "2024-11-20"),
    "event5_white_house_meeting": ("2025-02-15", "2025-03-10"),
}

# Convert to datetime for comparisons
event_windows = {k: (pd.to_datetime(v[0]), pd.to_datetime(v[1])) for k, v in event_windows.items()}
event_windows


{'event1_kyiv': (Timestamp('2022-02-20 00:00:00'),
  Timestamp('2022-03-20 00:00:00')),
 'event2_kherson': (Timestamp('2022-10-15 00:00:00'),
  Timestamp('2022-12-01 00:00:00')),
 'event3_stalemate': (Timestamp('2023-03-01 00:00:00'),
  Timestamp('2023-06-30 00:00:00')),
 'event4_trump_election': (Timestamp('2024-10-25 00:00:00'),
  Timestamp('2024-11-20 00:00:00')),
 'event5_white_house_meeting': (Timestamp('2025-02-15 00:00:00'),
  Timestamp('2025-03-10 00:00:00'))}

## 3. Subreddit Strategy (U.S.-focused vs Non-U.S.)

Reddit does not provide verified user location. To approximate U.S.-focused public opinion,
we label comments from subreddits that are strongly U.S.-centric (e.g., U.S. politics communities).

Important:
- We do **not** drop non-U.S. subreddits here.
- We keep all data and add a boolean column: `is_us_focused`.
- Downstream analysis can focus on `is_us_focused == True` while still allowing comparisons.


In [61]:
us_focused_subreddits = {
    "politics",
    "Conservative",
    "neoliberal",
    "WayOfTheBern",
    "AskReddit",
    "news",
    "worldnews",
    "geopolitics",
}

us_focused_subreddits


{'AskReddit',
 'Conservative',
 'WayOfTheBern',
 'geopolitics',
 'neoliberal',
 'news',
 'politics',
 'worldnews'}

## 4. Columns to Keep

The raw dataset contains many columns. We only keep what we need for analysis and reporting.

Minimum needed:
- comment timestamp (`created_time`)
- text (`self_text`)
- subreddit (`subreddit`)

Helpful metadata (if available):
- comment score, post title, post timestamp, etc.


In [63]:
cols_to_keep = [
    "comment_id",
    "created_time",
    "self_text",
    "subreddit",
    "score",
    "post_id",
    "post_title",
    "post_self_text",
    "post_created_time",
    "post_score",
]

cols_to_keep


['comment_id',
 'created_time',
 'self_text',
 'subreddit',
 'score',
 'post_id',
 'post_title',
 'post_self_text',
 'post_created_time',
 'post_score']

## 5. Cleaning Rules

This extraction step performs light cleaning:
- Drop rows with missing text or timestamp
- Remove obvious non-content placeholders: `[deleted]`, `[removed]`
- Add:
  - `dt` parsed timestamp column
  - `is_us_focused` boolean label for subreddit group

Heavy cleaning (tokenization, lemmatization, etc.) will happen in later notebooks.


In [65]:
def is_valid_text(s: str) -> bool:
    """Return True if text looks like a real comment (not empty/removed/deleted)."""
    if not isinstance(s, str):
        return False
    s = s.strip()
    if len(s) == 0:
        return False
    if s.lower() in {"[deleted]", "[removed]"}:
        return False
    return True


## 6. Chunked Extraction Loop

We iterate through the raw CSV in chunks and:
- keep selected columns
- parse timestamps
- label U.S.-focused subreddits
- filter each chunk into the relevant event windows
- append results to event CSVs


In [67]:
# Track how many rows written per event (for summary)
written_counts = defaultdict(int)

# Output paths for each event
event_paths = {event: os.path.join(output_dir, f"{event}.csv") for event in event_windows.keys()}

# Remove old outputs if they already exist (optional safety)
for p in event_paths.values():
    if os.path.exists(p):
        os.remove(p)

print("Output files will be created here:")
for event, p in event_paths.items():
    print(event, "->", p)

for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunksize, low_memory=False, on_bad_lines="skip")):
    print(f"\nProcessing chunk {i} ...")

    # Keep only columns that exist in this chunk (robustness)
    existing_cols = [c for c in cols_to_keep if c in chunk.columns]
    chunk = chunk[existing_cols].copy()

    # Parse datetime
    chunk["dt"] = pd.to_datetime(chunk["created_time"], errors="coerce")

    # Drop invalid dt or missing text/subreddit
    if "self_text" in chunk.columns:
        chunk = chunk[chunk["self_text"].apply(is_valid_text)]
    chunk = chunk.dropna(subset=["dt", "subreddit"])

    # Label U.S.-focused subreddits
    chunk["is_us_focused"] = chunk["subreddit"].isin(us_focused_subreddits)

    # For each event window, filter and append
    for event, (start, end) in event_windows.items():
        mask = (chunk["dt"] >= start) & (chunk["dt"] <= end)
        event_chunk = chunk.loc[mask].copy()

        if event_chunk.empty:
            continue

        out_path = event_paths[event]
        write_header = not os.path.exists(out_path)

        event_chunk.to_csv(out_path, mode="a", header=write_header, index=False)
        written_counts[event] += len(event_chunk)

    # Print progress every few chunks
    if i % 5 == 0:
        print("Progress so far:", dict(written_counts))

print("\nDone.")
print("Final rows written per event:")
for event in event_windows.keys():
    print(event, written_counts[event])


Output files will be created here:
event1_kyiv -> ../data/processed/event1_kyiv.csv
event2_kherson -> ../data/processed/event2_kherson.csv
event3_stalemate -> ../data/processed/event3_stalemate.csv
event4_trump_election -> ../data/processed/event4_trump_election.csv
event5_white_house_meeting -> ../data/processed/event5_white_house_meeting.csv

Processing chunk 0 ...
Progress so far: {}

Processing chunk 1 ...

Processing chunk 2 ...

Processing chunk 3 ...

Processing chunk 4 ...

Processing chunk 5 ...
Progress so far: {'event5_white_house_meeting': 65928}

Processing chunk 6 ...

Processing chunk 7 ...

Processing chunk 8 ...

Processing chunk 9 ...

Processing chunk 10 ...
Progress so far: {'event5_white_house_meeting': 241262, 'event4_trump_election': 47653}

Processing chunk 11 ...

Processing chunk 12 ...

Processing chunk 13 ...

Processing chunk 14 ...

Processing chunk 15 ...
Progress so far: {'event5_white_house_meeting': 241262, 'event4_trump_election': 359088}

Processing 

## 7. Validate Extracted Event Files

We now load each event CSV and compute basic statistics:
- total rows
- min/max date
- percent U.S.-focused
- top subreddits


In [69]:
def summarize_event_csv(path: str, top_n: int = 10):
    df = pd.read_csv(path, low_memory=False)
    df["dt"] = pd.to_datetime(df["created_time"], errors="coerce")

    total = len(df)
    dt_min = df["dt"].min()
    dt_max = df["dt"].max()

    us_pct = None
    if "is_us_focused" in df.columns:
        us_pct = df["is_us_focused"].mean() * 100

    top_subs = df["subreddit"].value_counts().head(top_n)

    return {
        "rows": total,
        "dt_min": dt_min,
        "dt_max": dt_max,
        "us_pct": us_pct,
        "top_subreddits": top_subs
    }


In [71]:
summaries = {}

for event, path in event_paths.items():
    if not os.path.exists(path):
        print(f"{event}: file not found (0 rows extracted)")
        continue

    s = summarize_event_csv(path, top_n=10)
    summaries[event] = s

    print(f"\n=== {event} ===")
    print("Rows:", s["rows"])
    print("Date range:", s["dt_min"], "→", s["dt_max"])
    if s["us_pct"] is not None:
        print(f"Percent U.S.-focused: {s['us_pct']:.2f}%")
    print("\nTop subreddits:")
    print(s["top_subreddits"])



=== event1_kyiv ===
Rows: 2940
Date range: 2022-02-20 12:52:49 → 2022-03-19 23:01:46
Percent U.S.-focused: 0.00%

Top subreddits:
subreddit
InvasionOfUkraine    1700
RussiaReplacement     664
RussiaDenies          306
GTAorRussia           185
UkraineWarFootage      69
UkrainePics            16
Name: count, dtype: int64

=== event2_kherson ===
Rows: 795
Date range: 2022-10-15 00:00:33 → 2022-11-30 17:18:46
Percent U.S.-focused: 0.00%

Top subreddits:
subreddit
RussiaDenies            214
GTAorRussia             166
volunteersForUkraine    146
ArtForUkraine           118
UkraineWarReports        89
InvasionOfUkraine        23
UkraineWarFootage        14
UkrainePics              12
LiveUkraine              11
RussiaReplacement         2
Name: count, dtype: int64

=== event3_stalemate ===
Rows: 9955
Date range: 2023-03-01 00:23:08 → 2023-06-29 23:17:35
Percent U.S.-focused: 0.00%

Top subreddits:
subreddit
WagnerVsRussia          4809
UkraineLosses           2626
volunteersForUkraine    

## 8. Save Extraction Summary 

This exports a lightweight summary table that can be referenced in the report.


In [73]:
summary_rows = []

for event, s in summaries.items():
    summary_rows.append({
        "event": event,
        "rows": s["rows"],
        "dt_min": s["dt_min"],
        "dt_max": s["dt_max"],
        "us_pct": s["us_pct"],
    })

summary_df = pd.DataFrame(summary_rows).sort_values("event")
summary_df


,event,rows,dt_min,dt_max,us_pct
0,event1_kyiv,2940,2022-02-20 12:52:49,2022-03-19 23:01:46,0.000000
1,event2_kherson,795,2022-10-15 00:00:33,2022-11-30 17:18:46,0.000000
2,event3_stalemate,9955,2023-03-01 00:23:08,2023-06-29 23:17:35,0.000000
3,event4_trump_election,359088,2024-10-25 00:00:06,2024-11-19 23:59:57,26.287150
4,event5_white_house_meeting,241262,2025-02-15 00:00:04,2025-03-08 12:23:23,32.528952


In [75]:
summary_out = os.path.join(output_dir, "event_extraction_summary.csv")
summary_df.to_csv(summary_out, index=False)
summary_out


'../data/processed/event_extraction_summary.csv'

## 9. Notes / Next Steps

Next notebooks will use the extracted event CSVs:

- `02_exploration.ipynb`  
  Quick inspection and sanity checks per event

- `03_sentiment.ipynb`  
  VADER sentiment analysis (overall + U.S.-focused comparison)

- `04_topics.ipynb`  
  Topic modeling per event window (LDA or similar)

The Streamlit dashboard will later load these per-event CSVs and allow interactive comparisons,
including U.S.-focused vs non-U.S. subreddit discussions.
